# Bandstructure Plotting for g-BC3

## Load the files

### Functions definition

In [ ]:
#### Functions definition

# Packages invoking
import numpy as np
import os

# Load work path
def get_dir(directory_path = None):
    if directory_path is None:
        current_dir = os.getcwd()
    else:
        current_dir = directory_path
    if os.path.exists(os.path.join(current_dir, "vasprun.xml")):
        print(f"vasprun.xml is found in the current directory: {current_dir}")
        return current_dir
    else:
        Dir = input("vasprun.xml is not found in the current directory. Please enter the directory manually:")
        if os.path.exists(os.path.join(Dir, "vasprun.xml")):
            print(f"vasprun.xml is found in the directory: {Dir}")
            return Dir
        else:
            print("vasprun.xml is not found in the specified directory.")
            return None

# Checks if the files are located in the working directory
def vasp_file_check(Dir):
    if(os.path.isfile(os.path.join(Dir,"EIGENVAL"))==False):
        print('Please check that the EIGENVAL file is located in your chosen directory')
        quit()
    if(os.path.isfile(os.path.join(Dir,"KPOINTS")) ==False):
        print('Please check that the KPOINTS file is located in your chosen directory')
        quit()
    if(os.path.isfile(os.path.join(Dir,"OUTCAR")) ==False):
        print('Please check that the KPOINTS file is located in your chosen directory')
        quit()
    else:
        return -1

# Read the locations and limits of the high symmetry lines
def get_high_symmetry_lines(Dir):
    file = open(os.path.join(Dir,"KPOINTS"),"r")
    KPOINTS = file.readlines()
    file.close()
    if(KPOINTS[2][0]!="l"):
        print("Make sure 'line' is on the third line of you KPOINTS file, you currently have: "+KPOINTS[2])
        quit()
    if(KPOINTS[3][0]=="c" or KPOINTS[3][0]=="k"):
        Format="cartesian"
    if(KPOINTS[3][0]=="r"):
        Format="reciprocal"
    with open(os.path.join(Dir, "KPOINTS"), "r") as kpoints_file:
        KPOINTS = [line for line in kpoints_file.readlines() if line.strip()]
    lines_number = int((len(KPOINTS)-4)/2)
    limits = [[] for i in range(lines_number)]; index = 0
    for i in range(4, len(KPOINTS)):
        if((i%2) == 0):
            limits[index].append(KPOINTS[i].split())
        if((i%2) != 0):
            limits[index].append(KPOINTS[i].split())
            index += 1
    return Format, lines_number, limits

# Read the fermi energy from the file `OUTCAR`
def get_fermi(Dir):
    file = open(os.path.join(Dir,"OUTCAR"),"r")
    OUTCAR = file.readlines()
    file.close
    for i in range(0, len(OUTCAR)):
        if("E-fermi" in OUTCAR[i]):
            fermi = OUTCAR[i].split()[2]
            print(f"The Fermi energy is: {str(fermi)} eV")
            return fermi

# Converts the reciprocal corrdinates to cartesian coordinates
def rec_to_cart(kL, DIR):
    Basis = input("Please input your k-space lattice basis format, such as fcc, bcc, hcp, sc: ")
    try:
        str(Basis)
    except ValueError:
        return rec_to_cart(kL,DIR)
    if(get_high_symmetry_lines(DIR)[0]=="rec"):
        if(Basis=="fcc" or Basis=="FCC"):
            B=np.array([[-1,1,1],[1,-1,1],[1,1,-1]])
            print("Your lattice is FCC")
        if(Basis=="bcc" or Basis=="BCC"):
            B=np.array([[0,1,1],[1,0,1],[1,1,0]])
            print("Your lattice is BCC")
        if(Basis=="hcp" or Basis=="HCP"):   # It may be worth automating the calculation of lengths "a" and "c" later.
            a=float(input("What is the side length of your lattice? "))
            c=float(input("What is the height of HCP lattice? "))
            B=np.array([[1,-1/np.sqrt(3),0],[0,2/np.sqrt(3),0],[0,0,a/c]])
            print("Your lattice is HCP")
        if(Basis=="sc" or Basis=="SC"):
            B=np.array([[1,0,0],[0,1,0],[0,0,1]])
            print("Your lattice is SC")

        for k in range(len(kL)):
            kL[k]=(kL[k][0]*B[0]+kL[k][1]*B[1]+kL[k][2]*B[2]).tolist()

        return kL
    else:
        return kL

def extract_kpoints_path(Dir):
    # Open and read the KPOINTS file
    with open(os.path.join(Dir,"KPOINTS"),"r") as file:
        lines = file.readlines()
    # Initialize the list to store k-points
    kpoints = []
    # Initialize the list to store labels
    labels = []
    # Flag to track if we are reading k-points
    reading_kpoints = False
    # Loop through the lines in the file starting from the fifth line
    for line in lines[4:]:
        # If the line is not empty
        if line.strip():
            # Split the line into kx, ky, kz, and label
            kx, ky, kz, label = line.split()
            # Append k-point information to the kpoints list
            kpoints.append({
                "kx": float(kx),
                "ky": float(ky),
                "kz": float(kz),
                "label": label})
            # Convert the label to LaTeX format and append to the labels list
            if label == "G":
                labels.append(r"$\Gamma$")
            else:
                labels.append(r"${}$".format(label))
        else:
            # Stop reading if an empty line is encountered after reading k-points
            if reading_kpoints:
                break
            reading_kpoints = True
    # Return the k-points and labels
    return kpoints, labels

### Plot specific graph

In [ ]:
#### Plot specific graph

# Packages invoking
import matplotlib.pyplot as plt

# Plot bandstructure and calculate the bandgap energy from the file `EIGENVAL`
def read_eigen(Dir):
    file = open(os.path.join(Dir,"EIGENVAL"),"r")
    EIGEN = file.readlines()
    file.close()
    kpoints_number = int(EIGEN[5].split()[1])
    bands_number = int(EIGEN[5].split()[2])
    high_symmetry_lines = int(get_high_symmetry_lines(Dir)[1])
    bands=[[] for n in range(kpoints_number)]
    kpoints=[[] for n in range(kpoints_number)]
    print(f"Total of {str(kpoints_number)} k-points.")
    index = 0
    for i in range(7,len(EIGEN)):
        if(i == 7):
            kpoints[index].append(float(EIGEN[i].split()[0]))
            kpoints[index].append(float(EIGEN[i].split()[1]))
            kpoints[index].append(float(EIGEN[i].split()[2]))
        if(i > 7 and len(EIGEN[i].split()) == 4):
            index+=1
            kpoints[index].append(float(EIGEN[i].split()[0]))
            kpoints[index].append(float(EIGEN[i].split()[1]))
            kpoints[index].append(float(EIGEN[i].split()[2]))
        if(i > 7 and len(EIGEN[i].split()) == 2):
            bands[index].append(float(EIGEN[i].split()[1]))
    kpoints = rec_to_cart(kpoints,Dir)
    print(f"The length of kpoints is {str(len(kpoints))}.")

    # Subtract fermi level
    bands = np.asarray(bands)
    bands = bands - float(get_fermi(Dir))
    bands = bands.tolist()

    # Calculate distance between points
    distance = []
    distance.append(0.0)
    for i in range(1,len(kpoints)):
        distance.append(abs(np.sqrt((kpoints[i][0]-kpoints[i-1][0])**2+(kpoints[i][1]-kpoints[i-1][1])**2+(kpoints[i][2]-kpoints[i-1][2])**2)))
    
    # High Symmetry line distances
    line_distance = np.add.reduceat(distance, np.arange(0, len(distance), int(kpoints_number/high_symmetry_lines)), dtype=float)
    for g in range(1,len(line_distance)):
        line_distance[g] = line_distance[g] + line_distance[g-1]
    line_distance=line_distance.tolist()
    line_distance.insert(0,0.0)
    print(line_distance)

    # Set the k-point distance of all lines to be utilised as the x-axis
    for k in range(1,len(distance)):
        distance[k]=distance[k]+distance[k-1]

    # Find the band gap
    low_dex = [0,0]
    high_dex = [0,0]
    Min =  999999
    Max = -Min

    for e in range(len(bands)):
        for f in range(bands_number):
            if(bands[e][f]<=0 and Max<bands[e][f]):
                Max=bands[e][f]
                low_dex[0] = e
                low_dex[1] = f
            if(bands[e][f] >0 and Min>bands[e][f]):
                Min = bands[e][f]
                high_dex[0] = e
                high_dex[1] = f
    
    # print(Min, Max, Min-Max, Low_dex, High_dex)
    bandgap = Min - Max
    print(f"The band gap of this system is: {str(bandgap)} eV")

    # Plot style
    params = {"text.usetex": False, "font.family": "serif", "mathtext.fontset": "cm", "axes.titlesize": 16, "axes.labelsize": 14, "figure.facecolor": "w"}
    plt.rcParams.update(params)

    # Graph settings
    kpoints_detail, bandstructure_xticks = extract_kpoints_path("3_Bandstructure_gamma/KPOINTS")

    # Ploting the graph
    for m in range(bands_number):
        plt.xticks(line_distance, bandstructure_xticks)
        plt.plot(distance, [band[m] for band in bands])
    

In [ ]:
# Run plotting program

Dir = get_dir("3_Bandstructure_gamma")

check = vasp_file_check(Dir)
if(check == -1):
    read_eigen(Dir)
